In [1]:
# Find the endmembers from the HR
#Hysime -> Nfindr  (Davies and Bouldin Index)
from skimage.metrics import peak_signal_noise_ratio as psnr, structural_similarity as ssim
import sklearn
from sklearn.decomposition import NMF
import cv2
import scipy.io as io
import numpy as np
import matplotlib.pyplot as plt
import os
from tools.hsi_to_rgb import hsi_to_rgb
from tqdm import tqdm, trange
from sklearn.metrics import davies_bouldin_score
from tools.hysime import *
from metrics import *

In [2]:
def np_normalize(array):
    return (array-np.min(array)) /(np.max(array) - np.min(array))

In [3]:
matfile = io.loadmat(r"Outputs/comparison/avivaface_sr03mm_vs_hr1mm.mat")
print(matfile.keys())

dict_keys(['__header__', '__version__', '__globals__', 'SR', 'HR', 'WL'])


In [4]:
HR = matfile["HR"]
h = HR.shape[0]
w = HR.shape[1]
ch = HR.shape[2]
HR = HR.reshape((h*w,ch))

SR = matfile["SR"]
SR = SR.reshape((h*w,ch))
print(f"HR shape: {HR.shape}")
print(f"SR shape: {SR.shape}")

HR shape: (80388, 1018)
SR shape: (80388, 1018)


In [ ]:
matfile_LR = io.loadmat(r"Datasets/aviva_1mm_aligned_r3.mat")
print(matfile_LR.keys())

In [ ]:
LR = matfile_LR["I_MS_LR"]
#LR = LR.reshape(-1,ch)
LRup = np.zeros(h,w,ch)

interpolations = [ cv2.INTER_NEAREST, cv2.INTER_LINEAR, cv2.INTER_CUBIC, cv2.INTER_LANCZOS4]
for i in range(ch):
    LRup[:,:,i] = cv2.resize(LR, (h,w), interpolation= cv2.INTER_LINEAR)

In [5]:
SRn = np_normalize(SR.reshape(h,w,ch))
HRn = np_normalize(HR.reshape(h,w,ch))
sam = SAM(SRn, HRn)
print(f"SAM: {sam}")


SAM: 1.660964582717293


In [6]:
ergas = ERGAS(SRn,HRn,3)
print(f"ERGAS: {ergas}")

ERGAS: 5.641272369683453


In [7]:
q2n, q2n_map = Q2n(SRn,HRn)
print(f"Q2n: {q2n}")

0
0
1
2
3
4
5
6
7
1
0
1
2
3
4
5
6
7
2
0
1
2
3
4
5
6
7
3
0
1
2
3
4
5
6
7
4
0
1
2
3
4
5
6
7
5
0
1
2
3
4
5
6
7
6
0
1
2
3
4
5
6
7
7
0
1
2
3
4
5
6
7
8
0
1
2
3
4
5
6
7
9
0
1
2
3
4
5
6
7
10
0
1
2
3
4
5
6
7
Q2n: 0.9772741144366915


In [8]:
# calculate the psnr and ssim between the images
pnsr_value = np.round(psnr(HRn, SRn, data_range=1),2)
ssim_value = np.round(ssim(HRn, SRn, data_range=1),2)
print(pnsr_value)
print(ssim_value)

28.4
0.84


In [ ]:
noise, noise_corr = est_noise(HR)
print(f"noise shape: {noise.shape}")
print(f"nosie_corr shape: {noise_corr.shape}")

In [ ]:
kf,Ek = hysime(HR,noise, noise_corr)
print(f"Ek shape: {Ek.shape}")
print(f"kf: {kf}")

In [ ]:
mix = HR @ Ek

In [ ]:
n_endmembers = 10

nfindr = NFINDR()
# endmembers = nfindr.extract(X_LR.reshape(h,w,ch), n_endmembers, maxit=100, normalize=False) 
endmembers = nfindr.extract(mix.reshape((h,w,kf)), n_endmembers, maxit=200, normalize=False)


In [ ]:
labels = 

In [ ]:
davies_bouldin_score

In [ ]:
for n_end in range(20):